In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train=pd.read_csv("/kaggle/input/titanic/train.csv")
test=pd.read_csv("/kaggle/input/titanic/test.csv")

In [3]:
train.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [6]:
def X_y(dataset):
    if 'Survived' in dataset.keys():
        X=dataset.drop('Survived',axis=1)
        y=dataset['Survived']
    else:
        X=dataset
        y=None
    return X,y

In [7]:
X_train,y_train=X_y(train)
X_test,y_test=X_y(test)

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [9]:
def transform_X(X):
    num_attribs = ["Age", "SibSp", "Parch", "Fare"]
    cat_attribs = ["Pclass", "Sex", "Embarked"]
    num_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ])
    cat_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("cat_encoder", OneHotEncoder(sparse_output=False)),
    ])
    preprocess_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", cat_pipeline, cat_attribs),
    ])
    X_tr= preprocess_pipeline.fit_transform(X[num_attribs + cat_attribs])
    scaler=StandardScaler()
    X_tr = scaler.fit_transform(X_tr)
    return X_tr

In [10]:
X_train=transform_X(X_train)

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X_train, X_valid, y_train, y_valid = train_test_split(
 X_train, y_train,test_size=0.025,random_state=42)

In [13]:
import tensorflow as tf
from tensorflow import keras

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [14]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=X_train[0].shape))
model.add(keras.layers.Dense(800, activation="relu"))
model.add(keras.layers.Dense(800, activation="relu"))
model.add(keras.layers.Dense(500, activation="relu"))
model.add(keras.layers.Dense(500, activation="relu"))
model.add(keras.layers.Dense(100, activation="relu"))
model.add(keras.layers.Dense(1, activation="sigmoid"))

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 12)                0         
                                                                 
 dense (Dense)               (None, 800)               10400     
                                                                 
 dense_1 (Dense)             (None, 800)               640800    
                                                                 
 dense_2 (Dense)             (None, 500)               400500    
                                                                 
 dense_3 (Dense)             (None, 500)               250500    
                                                                 
 dense_4 (Dense)             (None, 100)               50100     
                                                                 
 dense_5 (Dense)             (None, 1)                 1

In [16]:
model.compile(loss="binary_crossentropy",
 optimizer="sgd", metrics=["accuracy"])

In [17]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=500,
 restore_best_weights=True)
checkpoint_cb = keras.callbacks.ModelCheckpoint("my_keras_model.h5",
 save_best_only=True)

In [18]:
print("--> Checking for physical Tensorflow devices")
for device in tf.config.list_physical_devices():
    print(": {}".format(device.name))

--> Checking for physical Tensorflow devices
: /physical_device:CPU:0
: /physical_device:GPU:0


In [19]:
with tf.device('/GPU:0'):
    history = model.fit(X_train, y_train, epochs=5000,validation_data=(X_valid, y_valid),use_multiprocessing=True,callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/5000
28/28 [==============================] - 5s 17ms/step - loss: 0.6803 - accuracy: 0.6141 - val_loss: 0.6812 - val_accuracy: 0.4783
Epoch 2/5000
28/28 [==============================] - 0s 8ms/step - loss: 0.6553 - accuracy: 0.6313 - val_loss: 0.6678 - val_accuracy: 0.4783
Epoch 3/5000
28/28 [==============================] - 0s 8ms/step - loss: 0.6329 - accuracy: 0.6440 - val_loss: 0.6500 - val_accuracy: 0.4783
Epoch 4/5000
28/28 [==============================] - 0s 7ms/step - loss: 0.6114 - accuracy: 0.6809 - val_loss: 0.6297 - val_accuracy: 0.5652
Epoch 5/5000
28/28 [==============================] - 0s 8ms/step - loss: 0.5883 - accuracy: 0.7339 - val_loss: 0.6045 - val_accuracy: 0.7826
Epoch 6/5000
28/28 [==============================] - 0s 7ms/step - loss: 0.5652 - accuracy: 0.7961 - val_loss: 0.5784 - val_accuracy: 0.7826
Epoch 7/5000
28/28 [==============================] - 0s 7ms/step - loss: 0.5403 - accuracy: 0.8111 - val_loss: 0.5474 - val_accuracy: 0.8261
Epoch

In [20]:
model = keras.models.load_model("my_keras_model.h5")

In [21]:
def prediction(X,model=model):
    y=model.predict(X)
    y=np.where(y >0.5, 1,0).reshape(-1)
    return y

In [22]:
y_test_pred=prediction(transform_X(X_test))

14/14 [==============================] - 0s 2ms/step


In [23]:
def save_output(y_pred,passenger_id=test['PassengerId']):
    output_data=pd.DataFrame({'PassengerId': passenger_id,'Survived':y_pred})
    submission=output_data.to_csv("submission.csv",index=False)

In [24]:
save_output(y_test_pred)